In [1]:
# import sys
# sys.path.append('D:/WorkSpace\master_thesis/data_pubmed/')
# D:\WorkSpace\master_thesis\data_pmc\pmc_pickle

import gensim
import sklearn
import soynlp # 한국어 자연어 처리
import nltk # 영어 자연어 처리
from nltk import RegexpTokenizer
from nltk.corpus import stopwords

print('gensim  version = {}'.format(gensim.__version__))
print('sklearn version = {}'.format(sklearn.__version__))
print('soynlp  version = {}'.format(soynlp.__version__))
print('nltk  version = {}'.format(nltk.__version__))

gensim  version = 3.8.3
sklearn version = 0.23.1
soynlp  version = 0.0.493
nltk  version = 3.5


In [10]:
from os import listdir
from os.path import isfile, join
 
pubmedPath = 'D:/WorkSpace\master_thesis/data_pubmed/pubmed/pubmed-Etanercept-set.txt'

f = open(pubmedPath, 'rt', encoding='UTF8')
data = f.read()
f.close()

print(data)

PMID- 29482402
OWN - NLM
STAT- MEDLINE
DCOM- 20190624
LR  - 20190624
IS  - 1750-7448 (Electronic)
IS  - 1750-743X (Linking)
VI  - 10
IP  - 6
DP  - 2018 Mar 1
TI  - Etanercept for the treatment of rheumatoid arthritis.
PG  - 433-445
LID - 10.2217/imt-2017-0155 [doi]
AB  - Etanercept was the first specific anticytokine therapy approved for the treatment of 
      rheumatoid arthritis (RA). Its clinical efficacy and safety has been demonstrated by 
      several clinical trials in early as well as established disease. Etanercept, along 
      with other TNF inhibitors, have revolutionized management of RA and dramatically 
      improved disease activity, function, quality of life and mortality for these 
      patients. It is structurally distinct from other TNF inhibitors and thus has 
      desirable profiles for immunogenicity, drug survival and infection rate. With the 
      increasing number of etanercept biosimilars, there will likely be a resurgence of 
      their prescription. 

In [ ]:
class Node:
    def __init__(self, indented_line):
        self.children = []
        self.level = len(indented_line) - len(indented_line.lstrip())
        self.text = indented_line.strip()

    def add_children(self, nodes):
        childlevel = nodes[0].level
        while nodes:
            node = nodes.pop(0)
            if node.level == childlevel: # add node as a child
                self.children.append(node)
            elif node.level > childlevel: # add nodes as grandchildren of the last child
                nodes.insert(0,node)
                self.children[-1].add_children(nodes)
            elif node.level <= self.level: # this node is a sibling, no more children
                nodes.insert(0,node)
                return

    def as_dict(self):
        if len(self.children) > 1:
            return {self.text: [node.as_dict() for node in self.children]}
        elif len(self.children) == 1:
            return {self.text: self.children[0].as_dict()}
        else:
            return self.text

# Problem A [0 points]
def read_data(filenames):
    data = None
    # Begin CODE
    data = {}
    contents = []
    for filename in filenames:
        with gzip.open(filename,'rt') as f:
            contents.append(f.read())

    root = Node('root')
    root.add_children([Node(line) for line in contents[0].splitlines() if line.strip()])
    d = root.as_dict()['root']
    print(d[:50])
    # End CODE
    return data

In [18]:
import re

pmid = re.findall('^PMID- (.*)$', data, re.MULTILINE)
title = re.findall('^TI  - (.*)$', data, re.MULTILINE)

print(pmid)
# print(title)

['29482402', '30616405', '30411183', '30747501', '28940172', '28682112', '31469238', '28596043', '28837372', '28388820']


In [7]:
data

"PMID- 29482402\nOWN - NLM\nSTAT- MEDLINE\nDCOM- 20190624\nLR  - 20190624\nIS  - 1750-7448 (Electronic)\nIS  - 1750-743X (Linking)\nVI  - 10\nIP  - 6\nDP  - 2018 Mar 1\nTI  - Etanercept for the treatment of rheumatoid arthritis.\nPG  - 433-445\nLID - 10.2217/imt-2017-0155 [doi]\nAB  - Etanercept was the first specific anticytokine therapy approved for the treatment of \n      rheumatoid arthritis (RA). Its clinical efficacy and safety has been demonstrated by \n      several clinical trials in early as well as established disease. Etanercept, along \n      with other TNF inhibitors, have revolutionized management of RA and dramatically \n      improved disease activity, function, quality of life and mortality for these \n      patients. It is structurally distinct from other TNF inhibitors and thus has \n      desirable profiles for immunogenicity, drug survival and infection rate. With the \n      increasing number of etanercept biosimilars, there will likely be a resurgence of \n    

In [ ]:
# Text 파일 대상 Read
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer

document = []
for i in range(len(docLabels)):
    f = open(pubmedPath + docLabels[i], 'rt', encoding='UTF8')
    data = f.read()
    document.append(data)
    f.close()